In [46]:
import os.path
import glob
import nltk

'''
Reads the RST-DT corpus files and returns RST
The internal folder structure of the corpus must not be changed. 
By default the function assumes that the relative directory of the  corpus is "..\..\RST_corpus"
'''
def readCorpus(relDir="..\\..\\RST_corpus", dataClass="training"):
    
    #go to the corpus dir  
    if(dataClass == "training"):
        relDir +=  "\\data\\SumExp-Texts150\\"
        fileext = "*.formatted"
    else:
        raise ValueError("Value of 'dataClass' is incorrect. Select one of the following - 1)'training' 2)'test'")
 
    absDir = os.path.join(os.getcwd(), relDir)
    
    if(os.path.isdir(absDir)):
        print("Reading corpus from "+ absDir)
    else:
        raise ValueError("The dir " + absDir +" is incorrect or doesnot exist. Please check check the value set in 'relDir' ")
            
    files = os.path.join(absDir, fileext)
    
    #read each file
    dat = []
    for fname in glob.glob(files):
        with open(fname, 'r', encoding='utf-8') as f:
            
            # tokenize to words
            # clean and preprocess data here @Yulia example - remove the newline characters that are between the text
            dat = [nltk.word_tokenize(sent) for sent in f]

    return dat

corpus = readCorpus(relDir="..\\..\\RST_corpus")
# for sent in corpus:
#     print(" ".join(sent))

Reading corpus from D:\Study\Masters\Workspace\ANLP\ANLP_assignmentFinal\repo\RST-parser\..\..\RST_corpus\data\SumExp-Texts150\
